In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" #no annoyiing messages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [2]:
# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Importing Data from MNIST available on tf keras datasets

In [3]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32')/255.0
x_test = x_test.reshape(-1,28,28,1).astype('float32')/255.0

11493376/11490434 [==============================] - 3s 0us/step


### What we are trying to do
Our common thing when doing CNN is that we have:
1. Conv Layer
2. Batchnorm Layer
3. ReLU layer

So we create a class to make sure this becomes modular.
Like Pytorch.

In [4]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
        
    def call(self,input_tensor, training=False):
        #call method is forward method in pytorch
        x = self.conv(input_tensor)
        x = self.bn(x,training=training)
        x = tf.nn.relu(x)
        return x

In [5]:
model = keras.Sequential([
    CNNBlock(32),
    CNNBlock(64),
    CNNBlock(128),
    layers.Flatten(),
    layers.Dense(10)
])

In [6]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [7]:
model.fit(x_train,y_train,batch_size=64, epochs=3, verbose=2)
model.evaluate(x_test,y_test,batch_size=64,verbose=2)

Epoch 1/3
938/938 - 14s - loss: 0.5923 - accuracy: 0.9439
Epoch 2/3
938/938 - 10s - loss: 0.0906 - accuracy: 0.9818
Epoch 3/3
938/938 - 10s - loss: 0.0377 - accuracy: 0.9889
157/157 - 3s - loss: 0.0451 - accuracy: 0.9873


[0.04511084780097008, 0.9872999787330627]

### More things with Subclassing: ResNet like model
Now we will write a ResNET like model in subclassing format

In [8]:
class ResBlock(layers.Layer):
    def __init__(self,channels):
        super(ResBlock,self).__init__()
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        
        #for skip connections we need identity mapping
        self.identity_mapping = layers.Conv2D(channels[1],1,padding='same')
        # change number of channels to the second layer output
        
    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor,training=training)
        x = self.cnn2(x,training=training)
        
        #skip conncection
        x = self.cnn3(
            x + self.identity_mapping(input_tensor), training=training
        )
        # idenity maooing will return same no of channels as the second cnn block but has encoding
        # of only 1 Convolution Operation
        
        return self.pooling(x)      

In [9]:
class ResNet_Like(keras.Model):
    #we can inherit functions like Predict, Train, evaluate, and all layers
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32,32,64])
        self.block2 = ResBlock([128,128,256])
        self.block3 = ResBlock([128,256,512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)
    
    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        return self.classifier(x)
    
    def model(self):
        # show output shapes when subclassing
        x = keras.Input(shape=(28,28,1)) #give the input shape
        return keras.Model(inputs=[x], outputs=self.call(x))
        

In [10]:
model = ResNet_Like(num_classes=10)

In [11]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)
model.fit(x_train,y_train,batch_size=64, epochs=20, verbose=1)
model.evaluate(x_test,y_test,batch_size=64,verbose=2)

Epoch 1/20
938/938 [==============================] - 40s 42ms/step - loss: 0.0854 - accuracy: 0.9745
Epoch 2/20
938/938 [==============================] - 21s 22ms/step - loss: 0.0369 - accuracy: 0.9886
Epoch 3/20
938/938 [==============================] - 21s 22ms/step - loss: 0.0277 - accuracy: 0.9911
Epoch 4/20
938/938 [==============================] - 21s 22ms/step - loss: 0.0248 - accuracy: 0.9918
Epoch 5/20
938/938 [==============================] - 21s 22ms/step - loss: 0.0214 - accuracy: 0.9931
Epoch 6/20
938/938 [==============================] - 21s 22ms/step - loss: 0.0184 - accuracy: 0.9940
Epoch 7/20
938/938 [==============================] - 21s 22ms/step - loss: 0.0155 - accuracy: 0.9950
Epoch 8/20
938/938 [==============================] - 21s 23ms/step - loss: 0.0128 - accuracy: 0.9959
Epoch 9/20
938/938 [==============================] - 22s 23ms/step - loss: 0.0140 - accuracy: 0.9955
Epoch 10/20
938/938 [==============================] - 22s 23ms/step - loss: 0.011

[0.02252722717821598, 0.9937000274658203]

In [17]:
model.model().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
res_block_3 (ResBlock)       (None, 14, 14, 64)        28640     
_________________________________________________________________
res_block_4 (ResBlock)       (None, 7, 7, 256)         526976    
_________________________________________________________________
res_block_5 (ResBlock)       (None, 3, 3, 512)         1839744   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 2,400,490
Trainable params: 2,397,418
Non-trainable params: 3,072
_______________________________________________